In [1]:
import pytesseract
from PIL import Image
import pandas as pd
from pdf2image import convert_from_path
import os

In [2]:
from PIL import Image, ImageFilter, ImageEnhance

def preprocess_image(image: Image):
    # Convert to grayscale
    image = image.convert('L')
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)
    # Apply sharpening
    image = image.filter(ImageFilter.SHARPEN)
    return image

In [3]:
TESSERACT_OCR_PATH = os.getenv('TESSERACT_OCR_PATH')
POPPLER_PATH = os.getenv('POPPLER_PATH')

In [4]:
images = convert_from_path("test_data\S247247 RB Collection.pdf", dpi=300, poppler_path=POPPLER_PATH)
# images = convert_from_path("test_data\Boyang_Invoice_01.pdf", poppler_path=POPPLER_PATH)

In [5]:
# images[0].show()

In [6]:
# images[1].show()

In [7]:
all_processed_image_data = []
all_image_data = []

In [8]:
for image in images:
    image_processed = preprocess_image(image)
    processed_image_text = pytesseract.image_to_string(image_processed)
    processed_image_data = pytesseract.image_to_data(image_processed, output_type=pytesseract.Output.DATAFRAME)
    image_text = pytesseract.image_to_string(image)
    image_data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DATAFRAME)
    all_processed_image_data.append(processed_image_data)
    all_image_data.append(image_data)

In [9]:
all_image_data[0]

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,2546,3267,-1.000000,NaN
1,2,1,1,0,0,0,146,55,235,39,-1.000000,NaN
2,3,1,1,1,0,0,146,55,235,39,-1.000000,NaN
3,4,1,1,1,1,0,146,55,235,39,-1.000000,NaN
4,5,1,1,1,1,1,146,55,70,30,77.924156,Refine
...,...,...,...,...,...,...,...,...,...,...,...,...
266,2,1,8,0,0,0,2118,3060,226,30,-1.000000,NaN
267,3,1,8,1,0,0,2118,3060,277,32,-1.000000,NaN
268,4,1,8,1,1,0,2118,3060,226,30,-1.000000,NaN
269,5,1,8,1,1,1,2118,3060,226,30,88.687737,CONTINUED


In [38]:
for i in range(1, 2+1):
    globals()["page{}".format(i)] = 

1
2


In [11]:
page1 = all_image_data[0]

In [12]:
page1_processed = all_processed_image_data[0]

In [13]:
def find_column_location(data, name, conf_threshold):
    filtered_data = data[data.conf > conf_threshold]
    column_loc = filtered_data[filtered_data['text'].str.contains(name, case=False)]
    
    return column_loc

In [14]:
find_column_location(page1, 'qty', 40)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
116,5,1,6,1,8,1,120,1007,73,42,96.369591,Qty.


In [15]:
find_column_location(page1, 'item', 40)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
117,5,1,6,1,8,2,397,1016,87,27,96.607986,Item


In [16]:
find_column_location(page1, 'description', 40)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
118,5,1,6,1,8,3,1019,1023,211,43,96.393326,Description


In [17]:
find_column_location(page1, 'unit', 40)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
119,5,1,6,1,8,4,1674,1038,75,27,96.403191,Unit


In [18]:
find_column_location(page1, 'price', 40)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
120,5,1,6,1,8,5,1764,1039,90,27,96.537437,Price
121,5,1,6,1,8,6,1922,1038,181,43,90.280502,Disc.Price
122,5,1,6,1,8,7,2244,1044,162,42,90.455139,Ext.Price


In [19]:
def extract_column(data, name, conf_threshold, offset=0, width=0):
    column_loc = find_column_location(data, name, conf_threshold)
    column = data[(data.conf > conf_threshold) & (data.left >= column_loc.left.values[0]+offset) & (data.left < column_loc.left.values[0]+width)& (data.top >= column_loc.top.values[0])]
    
    return column

In [20]:
find_column_location(page1, 'qty', 40).left.values[0]

120

In [21]:
extract_column(page1, 'qty', 40, 0, 100)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
116,5,1,6,1,8,1,120,1007,73,42,96.369591,Qty.
124,5,1,6,1,9,1,175,1090,21,28,96.644592,4
132,5,1,6,1,11,1,172,1232,22,27,96.859718,4
147,5,1,6,1,13,1,149,1372,42,28,96.774406,12
160,5,1,6,1,15,1,168,1516,21,27,74.465485,4
171,5,1,6,1,17,1,166,1663,21,29,96.981422,4
186,5,1,6,1,19,1,165,1802,20,28,97.006500,6
197,5,1,6,1,21,1,163,1942,19,29,96.841354,8
209,5,1,6,1,23,1,159,2091,21,29,57.157650,4
224,5,1,6,1,25,1,158,2238,19,29,96.190353,2


In [22]:
extract_column(page1, 'item', 10, -180, 150)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
117,5,1,6,1,8,2,397,1016,87,27,96.607986,Item
125,5,1,6,1,9,2,275,1092,226,31,91.684013,PA-TERRIAN
133,5,1,6,1,11,2,273,1233,126,30,96.514763,DEENA
148,5,1,6,1,13,2,271,1374,236,32,91.208130,DEEP182022
161,5,1,6,1,15,2,266,1518,217,35,17.899231,JC18/20/22
172,5,1,6,1,17,2,266,1665,165,32,90.916084,LORRELL
187,5,1,6,1,19,2,263,1805,236,32,91.362640,SUP-BULK18
198,5,1,6,1,21,2,261,1944,236,35,92.236893,SUP-BULK22
210,5,1,6,1,23,2,259,2093,133,32,57.157650,U-ASIA
225,5,1,6,1,25,2,257,2240,137,31,92.101372,U-ELSA


In [23]:
extract_column(page1, 'description', 40, -17*30, 20*30)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
118,5,1,6,1,8,3,1019,1023,211,43,96.393326,Description
130,5,1,6,1,10,1,633,1150,200,30,84.673737,NATURAL-4
134,5,1,6,1,11,3,635,1237,84,26,96.541458,100%
135,5,1,6,1,11,4,745,1241,246,26,96.196281,UNPROCESSED
136,5,1,6,1,11,5,1015,1245,203,25,96.051003,BRAZILIAN
...,...,...,...,...,...,...,...,...,...,...,...,...
252,5,1,6,1,28,3,1132,2449,87,25,87.544357,BK-2
256,5,1,6,1,29,3,613,2528,87,25,86.084663,YAKI
257,5,1,6,1,29,4,728,2529,196,28,86.084663,"18""20""22"""
264,5,1,7,1,1,1,615,2584,61,26,95.420692,1-4


In [24]:
extract_column(page1, 'unit', 40, 0, 8*30)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
119,5,1,6,1,8,4,1674,1038,75,27,96.403191,Unit
120,5,1,6,1,8,5,1764,1039,90,27,96.537437,Price
126,5,1,6,1,9,3,1761,1117,105,28,95.663017,93.50
139,5,1,6,1,11,8,1759,1260,105,29,96.010681,95.00
152,5,1,6,1,13,6,1759,1401,103,29,95.421036,14.95
165,5,1,6,1,15,6,1757,1545,102,33,95.516045,14.95
178,5,1,6,1,17,8,1753,1693,104,29,95.906647,95.00
191,5,1,6,1,19,6,1775,1832,80,28,95.890366,8.00
202,5,1,6,1,21,6,1772,1973,81,33,96.902313,9.00
216,5,1,6,1,23,8,1746,2123,105,30,96.384560,81.95


In [25]:
extract_column(page1, 'disc', 40, 0, 8*30)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
121,5,1,6,1,8,6,1922,1038,181,43,90.280502,Disc.Price
127,5,1,6,1,9,4,2011,1121,105,28,96.323410,64.99
140,5,1,6,1,11,9,2010,1264,104,29,96.696411,59.99
153,5,1,6,1,13,7,2009,1405,102,30,96.201019,12.95
166,5,1,6,1,15,7,2007,1549,102,34,94.998779,12.95
179,5,1,6,1,17,9,2004,1697,103,30,95.912552,59.99
192,5,1,6,1,19,7,2025,1836,81,29,96.232590,7.00
203,5,1,6,1,21,7,2022,1979,81,32,96.308357,8.00
217,5,1,6,1,23,9,1997,2128,104,30,94.976433,74.99
232,5,1,6,1,25,9,1994,2273,105,28,96.164909,74.99


In [26]:
extract_column(page1, 'ext', 40, 0, 8*30)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
122,5,1,6,1,8,7,2244,1044,162,42,90.455139,Ext.Price
128,5,1,6,1,9,5,2290,1126,128,27,95.438156,259.96
141,5,1,6,1,11,10,2288,1269,129,30,95.921417,239.96
154,5,1,6,1,13,8,2288,1410,126,30,96.705666,155.40
167,5,1,6,1,15,8,2309,1553,103,37,96.716797,51.80
180,5,1,6,1,17,10,2282,1702,128,30,96.772583,239.96
193,5,1,6,1,19,8,2303,1841,105,29,95.625717,42.00
204,5,1,6,1,21,8,2302,1984,104,33,95.690399,64.00
218,5,1,6,1,23,10,2276,2133,128,30,96.226303,299.96
233,5,1,6,1,25,10,2276,2279,126,28,96.029816,149.98


In [95]:
if len(page1[page1.text=='CONTINUED']):
    page1 = page1[page1.top < page1[page1.text=='CONTINUED'].top.values[0]]

In [96]:
qty = extract_column(page1, 'qty', 10, 0, 100)
item = extract_column(page1, 'item', 10, -180, 150)
description = extract_column(page1, 'description', 40, -17*30, 20*30)
unit_price = extract_column(page1, 'unit', 10, 0, 8*30)
disc_price = extract_column(page1, 'disc', 10, 0, 8*30)
ext_price = extract_column(page1, 'ext', 10, 0, 8*30)

In [97]:
line_list = qty.line_num.unique()

In [98]:
line_list

array([ 8,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29], dtype=int64)

In [99]:
def join_words_in_line(data, line_num):
    desc = ''

    for text in data[data.line_num == line_num].text:
        if desc == '':
            desc = text
        else:
            desc = " ".join([desc, text])

    return desc

In [100]:
extracted_table = pd.DataFrame(columns=['line no', 'qty', 'item', 'description', 'unit price', 'disc price', 'ext price'])

In [102]:
if len(line_list) > 1:
    extracted_table['line no'] = line_list[1:]
    extracted_table['qty'] = qty.text.values[1:]
    extracted_table['item'] = item.text.values[1:]
    # extracted_table['description'] = description.text.values[1:]
    extracted_table['unit price'] = unit_price.text.values[2:]
    extracted_table['disc price'] = disc_price.text.values[1:]
    extracted_table['ext price'] = ext_price.text.values[1:]
    for i in range(len(extracted_table)):
        extracted_table.loc[i, 'description'] = join_words_in_line(description, extracted_table['line no'][i])
        extracted_table.loc[i, 'description'] += " #"
        if join_words_in_line(description, extracted_table['line no'][i]+1):
            extracted_table.loc[i, 'description'] += join_words_in_line(description, extracted_table['line no'][i]+1)
        else:
            extracted_table.loc[i, 'description'] += join_words_in_line(description, 1)

extracted_table

,line no,qty,item,description,unit price,disc price,ext price
0,9,4,PA-TERRIAN,#NATURAL-4,93.50,64.99,259.96
1,11,4,DEENA,100% UNPROCESSED BRAZILIAN HD LACE #NATURAL-2 ...,95.00,59.99,239.96
2,13,12,DEEP182022,"DEEP WAVE 18""20""22"" #1B-4 2-4 P1B30-4",14.95,12.95,155.40
3,15,4,JC18/20/22,JERRY CURL 18-20-22 #1-4,14.95,12.95,51.80
4,17,4,LORRELL,100% UNPROCESSED BRAZILIAN HD LACE #NATURAL-2 ...,95.00,59.99,239.96
5,19,6,SUP-BULK18,"SUPER BULK 18"" #1B-6",8.00,7.00,42.00
6,21,8,SUP-BULK22,"SUPER BULK 22"" #1-4 1B-4",9.00,8.00,64.00
7,23,4,U-ASIA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,81.95,74.99,299.96
8,25,2,U-ELSA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,86.95,74.99,149.98
9,27,4,U-VESA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,68.50,66.50,266.00


In [33]:
for i in range(len(extracted_table)):
    extracted_table.loc[i, 'description'] = join_words_in_line(description, extracted_table['line no'][i])
    extracted_table.loc[i, 'description'] += " #"
    if join_words_in_line(description, extracted_table['line no'][i]+1):
        extracted_table.loc[i, 'description'] += join_words_in_line(description, extracted_table['line no'][i]+1)
    else:
        extracted_table.loc[i, 'description'] += join_words_in_line(description, 1)

In [34]:
extracted_table.drop('line no', axis=1)

,qty,item,description,unit price,disc price,ext price
0,4,PA-TERRIAN,#NATURAL-4,93.50,64.99,259.96
1,4,DEENA,100% UNPROCESSED BRAZILIAN HD LACE #NATURAL-2 ...,95.00,59.99,239.96
2,12,DEEP182022,"DEEP WAVE 18""20""22"" #1B-4 2-4 P1B30-4",14.95,12.95,155.40
3,4,JC18/20/22,JERRY CURL 18-20-22 #1-4,14.95,12.95,51.80
4,4,LORRELL,100% UNPROCESSED BRAZILIAN HD LACE #NATURAL-2 ...,95.00,59.99,239.96
5,6,SUP-BULK18,"SUPER BULK 18"" #1B-6",8.00,7.00,42.00
6,8,SUP-BULK22,"SUPER BULK 22"" #1-4 1B-4",9.00,8.00,64.00
7,4,U-ASIA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,81.95,74.99,299.96
8,2,U-ELSA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,86.95,74.99,149.98
9,4,U-VESA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,68.50,66.50,266.00


In [88]:
page2 = all_image_data[1]

In [89]:
if len(page2[page2.text=='TOTAL']):
    page2 = page2[page2.top < page2[page2.text=='TOTAL'].top.values[0]]

In [90]:
qty = extract_column(page2, 'qty', 10, 0, 100)
item = extract_column(page2, 'item', 10, -180, 150)
description = extract_column(page2, 'description', 40, -17*30, 20*30)
unit_price = extract_column(page2, 'unit', 10, 0, 8*30)
disc_price = extract_column(page2, 'disc', 10, 0, 8*30)
ext_price = extract_column(page2, 'ext', 10, 0, 8*30)

In [91]:
line_list = qty.line_num.unique()

In [92]:
extracted_table = pd.DataFrame(columns=['line no', 'qty', 'item', 'description', 'unit price', 'disc price', 'ext price'])

In [93]:
if len(line_list) > 1:
    extracted_table['line no'] = line_list[1:]
    extracted_table['qty'] = qty.text.values[1:]
    extracted_table['item'] = item.text.values[1:]
    # extracted_table['description'] = description.text.values[1:]
    extracted_table['unit price'] = unit_price.text.values[2:]
    extracted_table['disc price'] = disc_price.text.values[1:]
    extracted_table['ext price'] = ext_price.text.values[1:]
    extracted_table


In [94]:
extracted_table

,line no,qty,item,description,unit price,disc price,ext price
